In [1]:
import pandas as pd

### Review county population dataset

In [5]:
df_cp = pd.read_parquet('./county_population.parquet')
df_cp.head()

,FIPS,year,population
index,,,
0,01001,2000,43872
1,01003,2000,141358
2,01005,2000,29035
3,01007,2000,19936
4,01009,2000,51181


### Review FIPS code (the key to merge)

In [24]:
df_fips = pd.read_parquet('./fips.parquet')
df_fips["CountyName"] = df_fips["CountyName"].str.replace(" County", "")
df_fips["CountyName"] = df_fips["CountyName"].str.lower()
df_fips

,FIPS,CountyName,StateName
0,01001,autauga,AL
1,01003,baldwin,AL
2,01005,barbour,AL
3,01007,bibb,AL
4,01009,blount,AL
...,...,...,...
3230,72153,yauco municipio,PR
3231,74300,midway islands,UM
3232,78010,st. croix island,VI
3233,78020,st. john island,VI


### Review opioid_data 

In [8]:
df_od = pd.read_parquet('./opioid_data_woFIPS.parquet')
df_od.head()

,year,BUYER_COUNTY,BUYER_STATE,opioid_converted_grams
0,2006,ABBEVILLE,SC,3136.215389
1,2006,ACADIA,LA,22401.553115
2,2006,ACCOMACK,VA,4073.851376
3,2006,ADA,ID,86952.634072
4,2006,ADAIR,IA,1129.828425


### Review drug overdosis death data

In [34]:
df_dod = pd.read_parquet('./drug_overdosis_death.parquet')
df_dod.rename(columns={"County Code": "FIPS","Year": "year"}, inplace=True)
df_dod["year"] = df_dod["year"].astype(int)
df_dod.head()

,FIPS,year,County,Deaths
0,01003,2003,"Baldwin County, AL",10.0
1,01003,2004,"Baldwin County, AL",18.0
2,01003,2005,"Baldwin County, AL",14.0
3,01003,2006,"Baldwin County, AL",11.0
4,01003,2007,"Baldwin County, AL",24.0


From the ouput, we can see use County code as key to merge. The opioid data has no county fips data, hence the first step is to merge od data with fips dataset.

### Generate County code for opioid data

In [27]:
# generate key for od data
df_od["county_key"] = df_od["BUYER_COUNTY"].str.lower() + ", " + df_od["BUYER_STATE"]
df_fips["county_key"] = df_fips["CountyName"] + ", " + df_fips["StateName"]
# generate key for fips data
df_od_fips = pd.merge(df_fips, df_od, left_on="county_key", right_on="county_key", how="right")
# filter columns
df_od_fips = df_od_fips[["FIPS","year","CountyName","StateName","opioid_converted_grams"]]
# show the results
df_od_fips.head()

,FIPS,year,CountyName,StateName,opioid_converted_grams
0,45001,2006,abbeville,SC,3136.215389
1,NaN,2006,NaN,NaN,22401.553115
2,51001,2006,accomack,VA,4073.851376
3,16001,2006,ada,ID,86952.634072
4,19001,2006,adair,IA,1129.828425


### Merge with County Population data by FIPS and year

In [28]:
df_od_fips_pop = pd.merge(df_od_fips, df_cp, on = ["FIPS","year"], how="left")
df_od_fips_pop.head()

,FIPS,year,CountyName,StateName,opioid_converted_grams,population
0,45001,2006,abbeville,SC,3136.215389,25440.0
1,NaN,2006,NaN,NaN,22401.553115,NaN
2,51001,2006,accomack,VA,4073.851376,38580.0
3,16001,2006,ada,ID,86952.634072,360869.0
4,19001,2006,adair,IA,1129.828425,7677.0


### Merge with overdose death data with FIPS and year

In [35]:
master = pd.merge(df_od_fips_pop, df_dod, on = ["FIPS","year"], how="left")
master.head()

,FIPS,year,CountyName,StateName,opioid_converted_grams,population,County,Deaths
0,45001,2006,abbeville,SC,3136.215389,25440.0,NaN,NaN
1,NaN,2006,NaN,NaN,22401.553115,NaN,NaN,NaN
2,51001,2006,accomack,VA,4073.851376,38580.0,NaN,NaN
3,16001,2006,ada,ID,86952.634072,360869.0,"Ada County, ID",32.0
4,19001,2006,adair,IA,1129.828425,7677.0,NaN,NaN


### drop Nan and manage column names

In [48]:
master = master.dropna()
master = master[["FIPS","year","County","opioid_converted_grams","population","Deaths"]]
master.head()

,FIPS,year,County,opioid_converted_grams,population,Deaths
3,16001,2006,"Ada County, ID",86952.634072,360869.0,32.0
8,08001,2006,"Adams County, CO",68317.538373,409532.0,62.0
26,45003,2006,"Aiken County, SC",28106.759579,150834.0,25.0
29,37001,2006,"Alamance County, NC",36524.619625,141472.0,10.0
30,06001,2006,"Alameda County, CA",376794.971387,1438193.0,182.0


In [47]:
# store values in parquet file and csv file
master.to_parquet("./master.parquet")
master.to_csv("./master.csv")

In [49]:
master.year.unique()

array([2006, 2007, 2008, 2009, 2010, 2011, 2012])